In [5]:
# Install, import libraries
import sys
!{sys.executable} -m pip install python-dotenv pandas pydap geopandas thredds_crawler

import geopandas as gpd
import pandas as pd
from pydap.client import open_url
from pydap.cas.urs import setup_session

import xarray as xr
import requests
import re

%load_ext dotenv
%dotenv
from os import environ

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
## Run this if you don't have a .nc file to work with

base_url = "https://oco2.gesdisc.eosdis.nasa.gov"
def list_urls(year):
    r = requests.get(f"https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.9r/{year}/catalog.xml")
    paths = re.findall("ID=\"(/opendap/hyrax/OCO2_L2_Lite_FP.9r/\d{4}/.*.nc4)\"", r.text)
    print(f"Found {len(paths)} files for {year}.")
    return paths

## Based on code from http://xarray.pydata.org/en/stable/io.html
def read_datasets(files, dim, transform_func=None):
    prog_ = 0
    session = setup_session(environ.get("EARTHDATA_USER"), environ.get("EARTHDATA_PASS"), check_url=base_url+files[0])
    def process_one_path(path,i):
        print(f"Loading dataset {i}/{len(files)}")
        # use a context manager, to ensure the file gets closed after use
        with xr.backends.PydapDataStore.open(path, session=session) as store:
            with xr.open_dataset(store) as ds:
                # transform_func should do some sort of selection or aggregation
                # aggregation
                if transform_func is not None:
                    ds = transform_func(ds)
                ds.load()
                return ds

    paths = sorted(files) # TODO use Glob
    datasets = [process_one_path(base_url+p,i) for i,p in enumerate(paths)]
    
    return xr.concat(datasets, dim)

# Method to reshape one dataset
def reshape_dataset(ds):
    return ds[[ 'sounding_id', 'latitude', 'longitude', 'time', 'xco2' ]]

# here we suppose we only care about the combined mean of each file;
# you might also use indexing operations like .sel to subset datasets
combined = read_datasets(list_urls(2017), dim='sounding_id', transform_func=reshape_dataset)

print(f"Done. Loaded {combined.nbytes * (2 ** -20)} MB into a dataset.")

combined.to_netcdf('2017_soundings.nc')

combined = combined.set_index(sounding_id=['latitude', 'longitude', 'time'])

Found 306 files for 2017.
Loading dataset 0/306
Loading dataset 1/306
Loading dataset 2/306
Loading dataset 3/306
Loading dataset 4/306
Loading dataset 5/306
Loading dataset 6/306
Loading dataset 7/306
Loading dataset 8/306
Loading dataset 9/306
Loading dataset 10/306
Loading dataset 11/306
Loading dataset 12/306
Loading dataset 13/306
Loading dataset 14/306
Loading dataset 15/306
Loading dataset 16/306
Loading dataset 17/306
Loading dataset 18/306
Loading dataset 19/306
Loading dataset 20/306
Loading dataset 21/306
Loading dataset 22/306
Loading dataset 23/306
Loading dataset 24/306
Loading dataset 25/306
Loading dataset 26/306
Loading dataset 27/306
Loading dataset 28/306
Loading dataset 29/306
Loading dataset 30/306
Loading dataset 31/306
Loading dataset 32/306
Loading dataset 33/306
Loading dataset 34/306
Loading dataset 35/306
Loading dataset 36/306
Loading dataset 37/306
Loading dataset 38/306
Loading dataset 39/306
Loading dataset 40/306
Loading dataset 41/306
Loading dataset 42

In [3]:
## Run this if you already have a file

combined = xr.open_dataset('2019_20_soundings.nc').set_index(sounding_id=['latitude', 'longitude', 'time'])
combined

<xarray.Dataset>
Dimensions:      (sounding_id: 63227343)
Coordinates:
  * sounding_id  (sounding_id) MultiIndex
  - latitude     (sounding_id) float64 -80.31 -82.25 -84.1 ... 44.03 44.07 44.08
  - longitude    (sounding_id) float64 -9.986 -20.53 -38.81 ... 42.01 42.03
  - time         (sounding_id) datetime64[ns] 2019-01-01T00:21:25.336017664 ... 2020-01-21T10:01:06.082991616
Data variables:
    xco2         (sounding_id) float32 ...
Attributes:
    identifier_product_doi:            10.5067/W8QGIYNKS3JC
    identifier_product_doi_authority:  http://dx.doi.org/
    filter_function:                   oco2_xco2_quality_flag_b9
    bc_function:                       oco2_bias_correct_b9
    warn_level_function:               oco2_warn_levels_b8
    Bias_Correction_land:              XCO2_Bias_Corrected = (XCO2_Raw + 0.90...
    Bias_Correction_oceanGL:           XCO2_Bias_Corrected = (XCO2_Raw + 0.24...
    Footprint_bias_land:               Assumed footprint biases in xco2 [ppm]...
    Footprint_bias_oceanGL:            Assumed footprint biases in xco2 [ppm]...
    Bias_Uncertainty:                  Bias correction parameter uncertaintie...
    Note_regarding_bias_correction:    There is some uncertainty in not only ...
    Platform:                          OCO-2
    Sensor:                            OCO-2
    title:                             ACOS L2 Lite Output
    BuildId:                           B8.1.00
    filtering_state:                   Filtered with oco2_lite_file_prefilter...
    creation_date:                     Mar 2019
    contact:                           Chris O'Dell: odell@atmos.colostate.edu

In [ ]:
unc = [-79.0558,35.9132]
tol = 1
result = None

try:
    result = combined.sel(
        longitude=slice(unc[0]-tol, unc[0]+tol),
        latitude=slice(unc[1]-tol, unc[1]+tol)
        )
except KeyError as e:
    result = f"Not found: {e}"
    
print(result)

In [ ]:
combined.where(combined.latitude <)

In [ ]:
base_url = "https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.9r/"
session = setup_session(environ.get("EARTHDATA_USER"), environ.get("EARTHDATA_PASS"), check_url=base_url+datasets[0])
# use a context manager, to ensure the file gets closed after use
store = xr.backends.PydapDataStore.open(base_url+datasets[0], session=session)
ds = xr.open_dataset(store)

ds = (ds[[ 'sounding_id', 'latitude', 'longitude', 'time', 'xco2' ]]
   .set_index(sounding_id=['latitude', 'longitude', 'time']))

In [ ]:
import time

unstacked = None;
for n in [100]:
    start = time.time()
    print(f"Unstacking {n} rows");
    unstacked = ds.isel(sounding_id=slice(0,n)).unstack()
    print(f"Elapsed for {n} time: {time.time()-start}s");

In [ ]:
ds.nbytes * (2 ** -20)

In [ ]:
unstacked.nbytes * (2 ** -20)

In [8]:
combined.nbytes * (2 ** -30)

1.6487814523279667